In [1]:
!pip install datasets
!pip install tqdm
!pip install pandas
!pip install rouge_score
!pip install bert_score
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d76be45b235a16bc14185eab21293b40fc426607c4d90150c71b95b1a2f3e280
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━

In [2]:
from datasets import load_dataset, load_metric
from tqdm import tqdm
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
samsum_dataset = load_dataset("samsum")
samsum_test_labels = samsum_dataset["test"]['summary']

labels = list(filter(lambda x: x.strip() != '', samsum_test_labels))

# print(samsum_test_labels)

In [13]:
# results_path = '/Users/valerio/Downloads/giuseppe_vanilla.txt'
results_path = '/content/drive/MyDrive/2_1_Deep_NLP/sick_data/tmp_result_emoji_keyword.txt'

# labels_path = '/Users/valerio/Downloads/giuseppe.txt'

with open(results_path, 'r') as file:
    sentences = file.read().split('\n')

predictions = list(filter(lambda x: x.strip() != '', sentences))

# with open(labels_path, 'r') as file:
#     sentences = file.read().split('\n')

# labels = list(filter(lambda x: x.strip() != '', sentences))


In [14]:
# print(len(labels))
print(len(samsum_test_labels))
print(len(predictions))

819
819


In [16]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(target, prediction, model_name='sentence-transformers/all-mpnet-base-v2'):
    # Load the model
    model = SentenceTransformer(model_name)

    # Encode the sentences
    embedding_target = model.encode(target, convert_to_tensor=True)
    embedding_prediction = model.encode(prediction, convert_to_tensor=True)

    # Reshape the embeddings for cosine similarity calculation
    embedding_target = embedding_target.reshape(1, -1)
    embedding_prediction = embedding_prediction.reshape(1, -1)

    # Compute cosine similarity
    similarity = cosine_similarity(embedding_target.cpu().detach().numpy(), embedding_prediction.cpu().detach().numpy())

    return similarity[0][0]



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=cdbefb6b9dedf7b1070130bb586ab4f8579058a9818ba5067dd814e45e2195ea
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [20]:
# '/content/drive/MyDrive/2_1_Deep_NLP/Enhanced-Abstractive-Chat-Summarization/utils/rouge.py'
# metric = load_metric('../utils/rouge.py')
metric = load_metric('/content/Enhanced-Abstractive-Chat-Summarization/utils/rouge.py')

bertscore_metric = load_metric("bertscore", lang='en', model_type='bert-base-uncased')

results_v2=[]
for i in tqdm(range(len(predictions))):

    r = metric.compute(predictions=[predictions[i]], references=[labels[i]])
    r = {key: value.mid.fmeasure * 100 for key, value in r.items()}

    bertscore_metric.add_batch(predictions=[predictions[i]], references=[labels[i]])
    bertscore_result = bertscore_metric.compute(lang='en', model_type='bert-base-uncased')
    bertscore_result = sum(bertscore_result['f1']) / len(bertscore_result['f1'])

    semantic_simarity_score = calculate_similarity(predictions[i], labels[i])
    r["similarity_score"] = semantic_simarity_score
    r['bert_score'] = bertscore_result

    results_v2.append(r)

  0%|          | 0/819 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

100%|██████████| 819/819 [19:11<00:00,  1.41s/it]


In [21]:
import pandas as pd

# Create a DataFrame
data = {'prediction': predictions, 'lables': labels, 'score': results_v2}
df = pd.DataFrame(data)

# Specify the Excel file path
excel_file_path = 'tmp_result_emoji_keyword.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print(f'Data has been saved to {excel_file_path}')

Data has been saved to tmp_result_emoji_keyword.xlsx


In [ ]:
from tqdm import tqdm
from transformers import pipeline, load_metric

# Load metrics
rouge_metric = load_metric('rouge.py')
bertscore_metric = load_metric('bertscore', lang='en', model_type='bert-base-uncased')

results = []

# Iterate through predictions and labels
for prediction, label in tqdm(zip(predictions, labels), total=len(predictions)):
    # Calculate ROUGE scores
    rouge_scores = rouge_metric.compute(predictions=[prediction], references=[label])
    rouge_scores = {key: value.mid.fmeasure * 100 for key, value in rouge_scores.items()}

    # Calculate BERTScore
    bertscore_metric.add_batch(predictions=[prediction], references=[label])
    bertscore_result = bertscore_metric.compute(lang='en', model_type='bert-base-uncased')
    bertscore_result = sum(bertscore_result['f1']) / len(bertscore_result['f1'])

    # Calculate semantic similarity
    semantic_similarity_score = calculate_similarity(prediction, label)

    # Add scores to the results dictionary
    rouge_scores['similarity_score'] = semantic_similarity_score
    rouge_scores['bert_score'] = bertscore_result

    # Append results to the list
    results.append(rouge_scores)


In [23]:
res_v2 = pd.DataFrame([labels, predictions, results_v2]).T
res_v2.columns = ['labels', 'predictions', 'results']
res_v2 = res_v2['results'].apply(pd.Series).merge(res_v2, left_index=True, right_index=True).drop('results', axis=1)

res_v2.to_excel('./tmp_result_emoji_keyword.xlsx')